# Klassifikation des Ziffern-Datensatzes

Ein berühmter Datensatz im Machine Learning Umfeld ist von komprimierten "Bildern" von handgeschriebenen Ziffern.

Pro Ziffer gibt es ungefähr 180 Samples.

In [ ]:
from sklearn.datasets import load_digits
digits = load_digits()
digits.data.shape

Ein Beispiel-Image aus dem Datensatz

In [ ]:
import matplotlib.pyplot as plt
plt.gray()
plt.matshow(digits.images[0])
plt.show()

Die Images sind als Vektor(64) entsprechend eines 8x8 Bildes mit 16 Graustufen abgelegt.

Wir werden jetzt die vorhandenen Labels ignorieren und versuchen den Datensatz über k-Means in 10 Klassen zu teilen.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=10, random_state=0)
clusters = kmeans.fit_predict(digits.data)
kmeans.cluster_centers_.shape

Das Ergebnis sind 10 Cluster in $\mathbb{R}^{64}$.
Die Cluster-Mittelpunkte sind Punkte im 64-dimensionalen Raum und quasi die "idealen" Ziffern.

Die schauen wir uns jetzt an:

In [ ]:
fig, ax = plt.subplots(2, 5, figsize=(8, 3))
centers = kmeans.cluster_centers_.reshape(10, 8, 8)
for axi, center in zip(ax.flat, centers):
    axi.set(xticks=[], yticks=[])
    axi.imshow(center, interpolation='nearest', cmap=plt.cm.binary)

Sieht ja gar nicht schlecht aus. Aber wir können das nachprüfen.

## Fehlerabschätzung
Wir vergleichen jetzt die Clusterzugehörigkeit mit dem echten
Label aus dem Datensatz:

Dazu matchen wir zunächst die Clusterzugehörigkeit mit dem echten Label `digits.target`, dazu
werden wir aus einem Array mit dem Cluster-Index ein Array mit der eigentlichen Ziffer machen: 

In [ ]:
from scipy.stats import mode
import numpy as np

# erzeuge ein Array von Nullen in der selben Form wie unsere k-Means Cluster
labels = np.zeros_like(clusters)

# hier nutzen wir
for i in range(10):
    # erzeuge Boolsches Array zum Filtern
    mask = (clusters == i)
    # und nutzen dieses um die entsprechenden Array-Elemente zu selektieren
    # Wir gehen davon aus, dass die am häufigsten auftretende Ziffer die "echte" Ziffer des Clusters ist
    labels[mask] = mode(digits.target[mask])[0]

print(labels, digits.target)

und erzeugen den Score:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(digits.target, labels)

knapp 80% Treffer!

Über die Confusion-Matrix können wir noch prüfen, wo die meisten Fehler entstanden sind:

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

mat = confusion_matrix(digits.target, labels)
sns.heatmap(mat.T, square=True, annot=True, fmt='d', cbar=False,
            xticklabels=digits.target_names,
            yticklabels=digits.target_names)
plt.xlabel('Ziffer gemäß Datensatz Label')
plt.ylabel('Ziffer gemäß k-Means');

Die Ziffer "8" ist schwierig, insbesondere mit "1" verwechselt, aber auch Überschneidungen mit "9" und "5".

"0", "6" und "7" sind fast perfekt erkannt.